# **Task 1**

---

## **EDA and Data Preprocessing**

The overall goal is to build a predictive model for customer chur. Specifically, this task will involve gathering relevant data, conducting EDA, and preparing the data set for model development.

The output is a comprehensive report detailing your data gathering, EDA, and data cleaning processes. The report should include:
- A summary of the data sets selected and the rationale for their inclusion

- Visualisations and statistical summaries from the EDA
- A description of the data cleaning and preprocessing steps taken
- The cleaned and preprocessed data set ready for model building


### **Exploratory data analysis**

First, we must load all sheets in order to better understand what we have and the statistical properties of the dataset.

In [14]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Ignore Warning
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load Excel file
file_path = "Data/Customer_Churn_Data_Large.xlsx"
xlsx = pd.ExcelFile(file_path)

In [17]:
# Load sheets into separate DataFrames
demographics = xlsx.parse("Customer_Demographics")
transactions = xlsx.parse("Transaction_History")
service = xlsx.parse("Customer_Service")
online = xlsx.parse("Online_Activity")
churn = xlsx.parse("Churn_Status")

Now we can start merging all the sheets by `CustomerID` column. To tackle the problem of **transactional format** of `Transaction_History` and `Customer_Service` sheets, I will aggregate the transactional data first using `.groupby('CustomerID')` and applied functions like _sum_, _count_, and _mean_.

This turned the transactional sheets into **customer-level summaries** (i.e., each row = one customer), making it possible to:

- Merge them safely with `Customer_Demographics`, `Online_Activity`, and `Churn_Status`.

- Do EDA with one observation per customer.

- Fit a proper model. 

In [18]:
# Aggregate transactional and service data first
trans_agg = transactions.groupby("CustomerID").agg({
    "AmountSpent": ["sum", "mean", "count"],
    "TransactionID": "nunique"
})
trans_agg.columns = ['TotalSpent', 'AvgSpent', 'NumTransactions', 'UniqueTransactions']

service_agg = service.groupby("CustomerID").agg({
    "InteractionID": "count",
    "ResolutionStatus": lambda x: (x == 'Resolved').mean()
})
service_agg.columns = ['ServiceInteractions', 'ResolutionRate']

# Merge all together
base_df = demographics.merge(trans_agg, on="CustomerID", how="left")
base_df = base_df.merge(service_agg, on="CustomerID", how="left")
base_df = base_df.merge(online, on="CustomerID", how="left")
base_df = base_df.merge(churn, on="CustomerID", how="left")

In [19]:
base_df.head()

,CustomerID,Age,Gender,MaritalStatus,IncomeLevel,TotalSpent,AvgSpent,NumTransactions,UniqueTransactions,ServiceInteractions,ResolutionRate,LastLoginDate,LoginFrequency,ServiceUsage,ChurnStatus
0,1,62,M,Single,Low,416.50,416.50000,1,1,1.0,1.0,2023-10-21,34,Mobile App,0
1,2,65,M,Married,Low,1547.42,221.06000,7,7,1.0,1.0,2023-12-05,5,Website,1
2,3,18,M,Single,Low,1702.98,283.83000,6,6,1.0,1.0,2023-11-15,3,Website,0
3,4,21,M,Widowed,Low,917.29,183.45800,5,5,2.0,0.5,2023-08-25,2,Website,0
4,5,21,M,Divorced,Medium,2001.49,250.18625,8,8,NaN,NaN,2023-10-27,41,Website,0


The `.head()` method allows us to view the first 5 rows in the dataset, this is useful for visual inspection of our columns

In [20]:
# Dataset dimensions and data types
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CustomerID           1000 non-null   int64         
 1   Age                  1000 non-null   int64         
 2   Gender               1000 non-null   object        
 3   MaritalStatus        1000 non-null   object        
 4   IncomeLevel          1000 non-null   object        
 5   TotalSpent           1000 non-null   float64       
 6   AvgSpent             1000 non-null   float64       
 7   NumTransactions      1000 non-null   int64         
 8   UniqueTransactions   1000 non-null   int64         
 9   ServiceInteractions  668 non-null    float64       
 10  ResolutionRate       668 non-null    float64       
 11  LastLoginDate        1000 non-null   datetime64[ns]
 12  LoginFrequency       1000 non-null   int64         
 13  ServiceUsage         1000 non-null

In [21]:
# Statistical summary of numerical features
base_df.describe()

,CustomerID,Age,TotalSpent,AvgSpent,NumTransactions,UniqueTransactions,ServiceInteractions,ResolutionRate,LastLoginDate,LoginFrequency,ChurnStatus
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,668.000000,668.000000,1000,1000.000000,1000.000000
mean,500.500000,43.267000,1267.074950,248.806511,5.054000,5.054000,1.500000,0.527695,2023-07-05 21:28:48,25.912000,0.204000
min,1.000000,18.000000,9.800000,9.800000,1.000000,1.000000,1.000000,0.000000,2023-01-01 00:00:00,1.000000,0.000000
25%,250.750000,30.000000,626.685000,203.734167,3.000000,3.000000,1.000000,0.000000,2023-04-08 00:00:00,13.750000,0.000000
50%,500.500000,43.000000,1232.880000,250.121458,5.000000,5.000000,1.500000,0.500000,2023-07-10 12:00:00,27.000000,0.000000
75%,750.250000,56.000000,1791.902500,295.023854,7.000000,7.000000,2.000000,1.000000,2023-10-01 06:00:00,38.000000,0.000000
max,1000.000000,69.000000,3386.040000,496.990000,9.000000,9.000000,2.000000,1.000000,2023-12-31 00:00:00,49.000000,1.000000
std,288.819436,15.242311,738.590013,79.372766,2.603433,2.603433,0.500375,0.421921,NaN,14.055953,0.403171


Let's check if missing values still exist in the dataset.

In [23]:
# Check for missing values
base_df.isnull().sum()

CustomerID               0
Age                      0
Gender                   0
MaritalStatus            0
IncomeLevel              0
TotalSpent               0
AvgSpent                 0
NumTransactions          0
UniqueTransactions       0
ServiceInteractions    332
ResolutionRate         332
LastLoginDate            0
LoginFrequency           0
ServiceUsage             0
ChurnStatus              0
dtype: int64

In [42]:
# Import encoding libraries
from sklearn.preprocessing import LabelEncoder

In [43]:
# Create a copy of the DataFrame for encoding
df_encoded = df.copy()
label_encoders = {}

In [44]:
# Loop through each column and apply Label Encoding
for col in df_encoded.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le

df_encoded.head()

,num_passengers,sales_channel,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete
0,2,0,2,262,19,7,2,0,61,1,0,0,5.52,0
1,1,0,2,112,20,3,2,0,61,0,0,0,5.52,0
2,2,0,2,243,22,17,6,0,36,1,1,0,5.52,0
3,1,0,2,96,31,4,2,0,61,0,0,1,5.52,0
4,2,0,2,68,22,15,6,0,36,1,0,1,5.52,0
